In [1]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue

from src import data 
from src import analyzer


import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn import linear_model
import cufflinks as cf
cf.go_offline()

In [63]:
##个股
#下午开盘
pm_13 = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/second_130000_sell_price.csv", index_col = 0)
pm_13 =pm_13[:'2021-06-18']
#收益率
mom = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/mom1.csv", index_col = 0)


In [4]:
mom = mom['2018':'2021-06-18']

In [3]:
am_open = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price.csv", index_col = 0)
am_open = am_open['2018':'2021-06-18']

In [4]:
pm_close =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price.csv", index_col = 0)
pm_close = pm_close['2018':'2021-06-18']

In [68]:
#整合数据，找到所有共同有的股票值
colum = [col for col in am_open.columns if col in list(pm_13.columns)]
len(colum)

4413

In [98]:
am_open_rv = am_open[colum]
pm_close_rv = pm_close[colum]
pm_13_rv = pm_13[colum]

In [107]:
d = am_open_rv.loc['2018-01-03']
y = d[d.isnull()].index
len(y)

1155

In [108]:
s = pm_13_rv.loc['2018-01-03']
x = s[s.isnull()].index
len(x)

1159

In [84]:
miss = [ix for ix in x if ix not in y]
miss

['000939.XSHE',
 '002323.XSHE',
 '002464.XSHE',
 '002501.XSHE',
 '002680.XSHE',
 '300462.XSHE',
 '300683.XSHE',
 '600610.XSHG']

In [14]:
mom_rv = mom[colum1]

In [9]:
#指数
bmk=pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/bmk_minute.csv", index_col = 0)
bmk_open = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/bmk_open.csv", index_col = 0)
bmk_close =pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/bmk_close.csv", index_col = 0)

In [10]:
#上午指数
bmk_am = bmk_open['2018':'2021-06-18']
#下午开盘指数
bmk_pm13_index = [s for s in bmk.index if '11:30:00' in s]
bmk_pm13 = bmk.loc[bmk_pm13_index]
bmk_pm13 = bmk_pm13['2018':'2021-06-18']
bmk_pm = bmk_close['2018':'2021-06-18']

In [11]:
#中证500开盘
bmk_am_zz = bmk_am['000905.XSHG']
bmk_pm13_zz = bmk_pm13['close_000905.XSHG']
bmk_pm_zz = bmk_pm['000905.XSHG']

In [12]:
dic1 = dict(zip(bmk_pm13_zz.index,bmk_am_zz.index))
bmk_pm13_zz = bmk_pm13_zz.rename(dic1,axis = 'index')

In [117]:
#个股

#上午的收益率
am_er =pm_13_rv/am_open_rv-1
# am_er = am_er.drop('2018-07-05 00:00:00',axis = 0)

#下午收益率
pm_er = pm_close_rv/pm_13_rv-1
# pm_er = pm_er.drop('2018-07-05 00:00:00',axis = 0)

In [118]:
#指数

#上午收益率
bmk_am_er = bmk_pm13_zz/bmk_am_zz-1
bmk_pm_er =bmk_pm_zz/bmk_pm13_zz-1

In [120]:
bmk_am_er

2018-01-02    0.006268
2018-01-03    0.008514
2018-01-04    0.002700
2018-01-05    0.002578
2018-01-08    0.002310
                ...   
2021-06-11   -0.003689
2021-06-15   -0.009860
2021-06-16   -0.010313
2021-06-17    0.007894
2021-06-18         NaN
Length: 840, dtype: float64

In [119]:
model =linear_model.LinearRegression()

In [101]:
def task(stock,q):
    i = 20
    a_er = am_er[stock]
    p_er = pm_er[stock]
    stats = np.full(len(a_er), np.nan)
    apm = np.full(len(stats),np.nan)
    while i<len(a_er):
        if len(a_er.iloc[i-20:i].dropna())!= 20:
            i+=1
        else:
            s_am = np.array(a_er.iloc[i-20:i])
            s_pm = np.array(p_er.iloc[i-20:i])
            s_bmk_a =np.array(bmk_am_er.iloc[i-20:i])
            s_bmk_p =np.array(bmk_pm_er.iloc[i-20:i])
#             s_mom = np.array(mom[i-20:i])
            model.fit(s_bmk_a.reshape(-1, 1),s_am)
            resid_a =s_am-model.predict(s_bmk_a.reshape(-1, 1))
#             print(resid_a)
            model.fit(s_bmk_p.reshape(-1, 1),s_pm)
            resid_p = s_pm-model.predict(s_bmk_p.reshape(-1, 1))
#             print(resid_p)
            resid = resid_a-resid_p
            stat = np.mean(resid)/(np.std(np.array(resid))/np.sqrt(20))
            stats[i]=stat
            i+=1
#     j =20
#     print(stats)
#     while j <len(apm):
#         if len(mom.iloc[j-20:j].dropna())!= 20 or len(stats.iloc[j-20:j].dropna())!= 20:
#             j+=1
#         else:
#             sta = np.array(apm.iloc[j-20:j])
#             mom = np.array(mom.iloc[j-20:j])
#             model.fit(sta.reshape(-1, 1),mom)
#             apm[j] = sta- model.predict(mom.reshape(-1, 1))
#             j+=1
    return q.put({stock:stats})

In [104]:
q = multiprocessing.Manager().Queue()
param = []
for stock in am_er.columns:
    param.append((stock,q))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(task,param)

#extract
elements = []
while q.qsize():
    elements.append(q.get())

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning:

invalid value encountered in double_scalars

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:23: Runti

In [105]:
d = pd.DataFrame(np.empty((len(am_er),am_er.shape[1]),dtype=object),columns = am_er.columns, index = am_er.index)
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    val = list(elements[i][stock])
    d[stock]=val

In [106]:
d.shift(1).to_csv("apm_v2.csv")

In [262]:
k=d.loc['2021-06-18']
#print(np.where(np.isnan(k)).shape)
print(k[k.isnull()])

000018.XSHE   NaN
000022.XSHE   NaN
000023.XSHE   NaN
000042.XSHE   NaN
000043.XSHE   NaN
               ..
688425.XSHG   NaN
001207.XSHE   NaN
688216.XSHG   NaN
001208.XSHE   NaN
688690.XSHG   NaN
Name: 2021-06-18, Length: 434, dtype: float64


In [158]:
#每只股票对动量因子做横截面切割取residual
def delete_mom(date):
    apm = np.full(len(d), np.nan)
    s_apm = d.loc[date].values
    print(np.where(np.isnan(s_apm)))
    s_mom = df.loc[date].values
    print(np.where(np.isnan(s_mom)))
    clf.fit(s_mom.reshape(-1,1), s_apm)
    resid = s_apm - clf.predict(s_mom.reshape(-1,1))
    print(resid.shape)
    return resid
    

In [159]:
delete_mom('2021-06-18')

(array([  14,   18,   19,   35,   36,   82,   88,   92,  102,  110,  131,
        160,  161,  175,  189,  211,  221,  228,  247,  281,  289,  294,
        313,  316,  335,  389,  417,  423,  438,  445,  448,  484,  490,
        518,  524,  528,  536,  537,  538,  542,  543,  587,  588,  609,
        616,  654,  664,  666,  686,  713,  724,  725,  745,  747,  788,
        798,  801,  802,  819,  820,  822,  824,  842,  883,  910,  915,
        938,  942,  961,  964,  967,  974,  978, 1033, 1039, 1050, 1068,
       1098, 1144, 1156, 1170, 1173, 1175, 1177, 1180, 1190, 1208, 1209,
       1229, 1241, 1250, 1256, 1266, 1267, 1273, 1284, 1307, 1316, 1319,
       1327, 1329, 1338, 1414, 1417, 1447, 1455, 1485, 1486, 1527, 1546,
       1560, 1581, 1612, 1613, 1671, 1776, 1811, 1816, 1825, 1845, 1855,
       1863, 1884, 1899, 1902, 1926, 1939, 1944, 1961, 1974, 1987, 1988,
       2001, 2004, 2008, 2012, 2044, 2056, 2062, 2074, 2084, 2089, 2097,
       2100, 2105, 2110, 2113, 2118, 2136, 2143, 2

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
apm_raw =pd.DataFrame(np.empty((len(d),d.shape[1]),dtype=object),columns = d.columns, index = d.index)
for dates in d.index:
    raw =delete_mom(dates)
    apm_raw = raw

In [140]:
#计算过去20天的收益率
def ret(stock, p):
    i =20
    m = mom_rv[stock]
    s_mom = np.full(len(mom_rv),np.nan)
    while i<len(mom_rv):
        if len(m[i-20:i].dropna()) != 20:
            i+=1
        else:
            s = m[i-20:i]
            s_mom[i] =s.mean()
            i+=1
    return p.put({stock:s_mom})

In [141]:
p = multiprocessing.Manager().Queue()
param1 = []
for stock in mom_rv.columns:
    param1.append((stock,p))
# print(param)
#task
with multiprocessing.Pool(processes =4) as pool:
    pool.starmap(ret,param1)

#extract
elements1 = []
while p.qsize():
    elements1.append(p.get())

In [142]:
df = pd.DataFrame(np.empty((len(mom_rv),mom_rv.shape[1]),dtype=object),columns = mom_rv.columns, index = mom_rv.index)
for i in range(len(elements1)):
    stock = list(elements1[i].keys())[0]
    val = list(elements1[i][stock])
    df[stock]=val